# 빅콘테스트 퓨처스리그 : KBO 정규시즌 팀별 승률, 타율, 방어율 예측

2020.08.08 토요일 김채형

# Preprocessing

## 파생변수 생성하기 - 타율, 출루율, 장타율, OPS (with batter_tidy.csv)

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/Users/kimchaehyeong/Documents/BIGCONTEST/Baseball_ChilliShrimp/data/batter_tidy.csv')

In [3]:
data

,year,month,T_ID,P_ID,PA,AB,RBI,RUN,HIT,H2,...,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,BAT_ORDER,HOME,AWAY
0,2016,4,HH,60404,12,12,1,0,2,0,...,2,0,2,1,0,0,2,9,3,4
1,2016,4,HH,60667,10,9,2,1,3,1,...,2,3,0,3,0,0,0,8,4,4
2,2016,4,HH,60757,0,0,0,0,0,0,...,0,0,0,1,0,0,0,9,1,0
3,2016,4,HH,60805,1,0,0,1,0,0,...,0,0,0,0,0,2,0,2,2,0
4,2016,4,HH,61700,36,31,2,6,6,1,...,3,3,3,3,0,0,3,8,9,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6929,2020,7,WO,69306,5,5,0,2,1,1,...,0,0,1,1,0,2,0,1,4,2
6930,2020,7,WO,75125,67,57,15,11,15,1,...,0,0,3,2,3,3,0,4,8,9
6931,2020,7,WO,78168,78,68,4,15,20,3,...,0,0,3,2,3,3,0,1,8,9
6932,2020,7,WO,79365,43,41,5,4,10,3,...,0,0,2,2,0,2,0,5,6,5


In [4]:
data.columns

Index(['year', 'month', 'T_ID', 'P_ID', 'PA', 'AB', 'RBI', 'RUN', 'HIT', 'H2',
       'H3', 'HR', 'SB', 'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'ERR',
       'LOB', 'P_AB_CN', 'P_HIT_CN', 'VS_T_ID_HH', 'VS_T_ID_HT', 'VS_T_ID_KT',
       'VS_T_ID_LG', 'VS_T_ID_LT', 'VS_T_ID_NC', 'VS_T_ID_OB', 'VS_T_ID_SK',
       'VS_T_ID_SS', 'VS_T_ID_WO', 'BAT_ORDER', 'HOME', 'AWAY'],
      dtype='object')

In [5]:
df = data.copy()

### 타율 (AVG)
 
타율 = 안타수 / 타수
- 안타 = HIT
- 타수 = AB

In [6]:
# 타율
df['AVG'] = df['HIT'] / df['AB']

In [7]:
df[['AB', 'HIT', 'AVG']]

,AB,HIT,AVG
0,12,2,0.166667
1,9,3,0.333333
2,0,0,NaN
3,0,0,NaN
4,31,6,0.193548
...,...,...,...
6929,5,1,0.200000
6930,57,15,0.263158
6931,68,20,0.294118
6932,41,10,0.243902


In [8]:
sum(np.isnan(df['AVG']))

709

In [9]:
df['AVG'] = df['AVG'].fillna(0)

In [10]:
df[['AB', 'HIT', 'AVG']]

,AB,HIT,AVG
0,12,2,0.166667
1,9,3,0.333333
2,0,0,0.000000
3,0,0,0.000000
4,31,6,0.193548
...,...,...,...
6929,5,1,0.200000
6930,57,15,0.263158
6931,68,20,0.294118
6932,41,10,0.243902


In [11]:
sum(np.isnan(df['AVG']))

0

### 출루율 (OBP)

출루율 = (안타+볼넷+사구) / (타수+볼넷+사구+희생플라이)
- 안타 = HIT
- 볼넷 = BB
- 사구 = HP
- 타수 = AB
- 희생플라이 = SF

In [12]:
# 출루율
df['OBP'] = (df['HIT'] + df['BB'] + df['HP']) / (df['AB'] + df['BB'] + df['HP'] + df['SF'])

In [13]:
df[['HIT', 'BB', 'HP', 'AB', 'SF', 'OBP']]

,HIT,BB,HP,AB,SF,OBP
0,2,0,0,12,0,0.166667
1,3,1,0,9,0,0.400000
2,0,0,0,0,0,NaN
3,0,1,0,0,0,1.000000
4,6,4,1,31,0,0.305556
...,...,...,...,...,...,...
6929,1,0,0,5,0,0.200000
6930,15,9,0,57,1,0.358209
6931,20,9,0,68,1,0.371795
6932,10,1,1,41,0,0.279070


In [14]:
sum(np.isnan(df['OBP']))

689

In [15]:
df['OBP'] = df['OBP'].fillna(0)

In [16]:
sum(np.isnan(df['OBP']))

0

### 장타율 (SLG)

SLG = (단타개수x1 + 2루타개수x2 + 3루타개수x3 + 홈런개수x4) / 타수
- 단타 = HIT-H2-H3-HR
- 2루타 = H2
- 3루타 = H3
- 홈런 = HR
- 타수 = AB

In [17]:
# 장타율
df['SLG'] = ((df['HIT'] - df['H2'] - df['H3'] - df['HR'])*1 + df['H2']*2 + df['H3']*3 + df['HR']*4) / df['AB']

In [18]:
sum(np.isnan(df['SLG']))

709

In [19]:
df['SLG'] = df['SLG'].fillna(0)

In [20]:
sum(np.isnan(df['SLG']))

0

### OPS

OPS = 출루율 + 장타율

In [21]:
df['OPS'] = df['OBP'] + df['SLG']

In [22]:
df[['OBP', 'SLG', 'OPS']]

,OBP,SLG,OPS
0,0.166667,0.166667,0.333333
1,0.400000,0.444444,0.844444
2,0.000000,0.000000,0.000000
3,1.000000,0.000000,1.000000
4,0.305556,0.322581,0.628136
...,...,...,...
6929,0.200000,0.400000,0.600000
6930,0.358209,0.596491,0.954700
6931,0.371795,0.441176,0.812971
6932,0.279070,0.390244,0.669314


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6934 entries, 0 to 6933
Data columns (total 42 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        6934 non-null   int64  
 1   month       6934 non-null   int64  
 2   T_ID        6934 non-null   object 
 3   P_ID        6934 non-null   int64  
 4   PA          6934 non-null   int64  
 5   AB          6934 non-null   int64  
 6   RBI         6934 non-null   int64  
 7   RUN         6934 non-null   int64  
 8   HIT         6934 non-null   int64  
 9   H2          6934 non-null   int64  
 10  H3          6934 non-null   int64  
 11  HR          6934 non-null   int64  
 12  SB          6934 non-null   int64  
 13  CS          6934 non-null   int64  
 14  SH          6934 non-null   int64  
 15  SF          6934 non-null   int64  
 16  BB          6934 non-null   int64  
 17  IB          6934 non-null   int64  
 18  HP          6934 non-null   int64  
 19  KK          6934 non-null  

In [24]:
# 파일 저장
df.to_csv('batter_tidy_AVG_OBP_SLG_OPS.csv')